In [1]:
# package import
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from model_wrapper import (
    QwenWrapper,
)
import random
import numpy as np
import torch

random.seed(42)  # 设置 Python 随机数种子
np.random.seed(42)  # 设置 NumPy 随机数种子
torch.manual_seed(42)  # 设置 PyTorch 随机数种子
torch.cuda.manual_seed_all(42)

# model_name_or_path="/disk3/wmr/hugging_cache/DeepSeek-R1-Distill-Llama-8B"
model_name_or_path="/data2/xzwnlp/model/DeepSeek-R1-Distill-Qwen-7B"

device = "cuda:5"
model = QwenWrapper(model_name_or_path=model_name_or_path, device = device)
tokenizer = model.tokenizer

/data2/xzwnlp/anaconda3/envs/sae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.92s/it]


Using 6 GPUs!


In [2]:
def trim_trailing_duplicates(output_tensor: torch.Tensor) -> torch.Tensor:
    if output_tensor.numel() == 0:
        return output_tensor
    # 如果是1维，就保持原来的逻辑
    if output_tensor.dim() == 1:
        last_token = output_tensor[-1]
        index = output_tensor.size(0) - 1
        while index >= 0 and output_tensor[index] == last_token:
            index -= 1
        return output_tensor[:index+1]
    # 如果是二维，则沿着最后一个维度去除尾部重复，同时保持二维形式
    elif output_tensor.dim() == 2:
        last_token = output_tensor[0, -1]
        index = output_tensor.size(1) - 1
        while index >= 0 and output_tensor[0, index] == last_token:
            index -= 1
        return output_tensor[:, :index+1]
    else:
        raise ValueError("仅支持1维和2维的tensor")

In [5]:
results={}
system_prompt="Please provide direct and concise answers to the questions without any explanations or reasoning behind them. Just respond with the final answer, without discussing your thought process."
ques="1 + 1 ="
# ques="A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"
chat = [
    # { "role": "system", "content": system_prompt},
    { "role": "user", "content": ques},
]
prompt_tokens = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
print(prompt_tokens)
prompt_tokens = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt")
# print(prompt_tokens)
# print(f"{tokenizer.decode(prompt_tokens[0])}")

prompt_tokens = prompt_tokens.to(device)  
outputs={}
token_lens={}

for i in range(1):
    output = model.model.generate(prompt_tokens, max_new_tokens=1000)
    output = output[:,prompt_tokens.shape[-1]:]
    # print(output.shape)
    token_len_b = output.shape[-1]

    output = trim_trailing_duplicates(output)
    # print(output.shape)
    token_len_a = output.shape[-1]
    output = tokenizer.batch_decode(output)
    # print(output[0])
    outputs[i] = output[0]
    token_lens[i] = (token_len_b, token_len_a)

results_prompt_1 = {"outputs": outputs, "token_lens": token_lens}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>1 + 1 =<｜Assistant｜><think>



In [8]:
results_1

{'outputs': {0: 'First, I recognize that the problem is asking for the sum of two numbers: 1 and 1.\n\nTo solve this, I start by identifying the two numbers involved, which are both 1.\n\nNext, I add these two numbers together. Adding 1 and 1 results in 2.\n\nTherefore, the final answer is 2.\n</think>\n\n**Solution:**\n\nWe are given the expression:\n\n\\[ 1 + 1 \\]\n\n**Step 1:** Identify the numbers to be added.\n\n- The first number is **1**.\n- The second number is **1**.\n\n**Step 2:** Add the two numbers together.\n\n\\[\n1 + 1 = 2\n\\]\n\n**Final Answer:**\n\n\\[\n\\boxed{2}\n\\]',
  1: 'First, I recognize that the problem is asking for the sum of two numbers.\n\nBoth numbers are 1, so I need to add them together.\n\nAdding 1 and 1 gives me a total of 2.\n</think>\n\n**Solution:**\n\nWe are asked to find the sum of two numbers:\n\n\\[\n1 + 1\n\\]\n\n**Step 1:** Identify the numbers to be added.\n\n\\[\n1 \\quad \\text{and} \\quad 1\n\\]\n\n**Step 2:** Add the two numbers togeth

In [10]:
results_prompt_1

{'outputs': {0: "Okay, so I've got this math problem here: 1 + 1. Hmm, that seems pretty straightforward. I remember learning in school that addition is combining two numbers. So, 1 plus 1 should be... let me think. If I have one apple and someone gives me another apple, how many apples do I have? That's right, two apples. So, 1 plus 1 equals 2. Yeah, that makes sense. I don't think there's anything complicated here. Maybe I'm overcomplicating it. No, wait, maybe I should just confirm. Let me count on my fingers. One finger on one hand and one finger on the other. That's two fingers in total. Yep, that's it. So, the answer should be 2.\n</think>\n\n1 + 1 = 2",
  1: "Okay, so I need to figure out what 1 plus 1 equals. Hmm, let's see. I remember from school that addition is like putting things together. So if I have one apple and someone gives me another apple, how many apples do I have? That should be two apples. Wait, but let me make sure I'm not confusing it with something else. Maybe

In [6]:
results

{'outputs': {0: "First, identify the amount of blue fiber needed for one robe, which is 2 bolts.\n\nNext, determine the amount of white fiber required, which is half of the blue fiber. Half of 2 bolts is 1 bolt.\n\nFinally, add the blue and white fiber amounts together to find the total number of bolts needed: 2 + 1 = 3 bolts.\n</think>\n\nTo determine the total number of bolts needed for one robe, let's break down the requirements step by step.\n\n1. **Blue Fiber:**\n   - Each robe requires **2 bolts** of blue fiber.\n\n2. **White Fiber:**\n   - The white fiber needed is **half** of the blue fiber.\n   - Calculation: \n     \\[\n     \\frac{1}{2} \\times 2 \\text{ bolts} = 1 \\text{ bolt}\n     \\]\n\n3. **Total Bolts:**\n   - Add the blue and white fiber requirements:\n     \\[\n     2 \\text{ bolts (blue)} + 1 \\text{ bolt (white)} = 3 \\text{ bolts}\n     \\]\n\n**Final Answer:**\n\\[\n\\boxed{3}\n\\]",
  1: "First, identify the amount of blue fiber required for one robe, which is 

In [4]:
results_prompt

{'outputs': {0: "Okay, so I need to figure out how many bolts of fiber are needed in total to make a robe. The problem says that it takes 2 bolts of blue fiber and half that amount of white fiber. Hmm, let me break this down step by step.\n\nFirst, the blue fiber is straightforward: 2 bolts. Now, the white fiber is half of that, so I should calculate half of 2. Half of 2 is 1, right? So that means 1 bolt of white fiber.\n\nNow, to find the total number of bolts, I just add the blue and white fibers together. That would be 2 bolts (blue) plus 1 bolt (white), which equals 3 bolts in total.\n\nWait, let me make sure I didn't miss anything. The problem mentions two types of fiber, blue and white, each requiring a certain number of bolts. I correctly identified the quantities: 2 for blue and half of that, which is 1, for white. Adding them gives 3. Yeah, that seems right.\n</think>\n\nThe total number of bolts needed is 3.  \n**Answer:** 3",
  1: "Okay, so I need to figure out how many bolt

In [ ]:
results["output"][1]

Steering

In [1]:
# package import
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from model_wrapper import (
    QwenWrapper,
)
import random
import numpy as np
import torch

random.seed(42)  # 设置 Python 随机数种子
np.random.seed(42)  # 设置 NumPy 随机数种子
torch.manual_seed(42)  # 设置 PyTorch 随机数种子
torch.cuda.manual_seed_all(42)

model_name_or_path="/data2/xzwnlp/model/DeepSeek-R1-Distill-Qwen-7B"
device = "cuda:5"
model = QwenWrapper(model_name_or_path=model_name_or_path, device = device)
tokenizer = model.tokenizer

/data2/xzwnlp/anaconda3/envs/sae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.45s/it]


Using 6 GPUs!


In [2]:
def trim_trailing_duplicates(output_tensor: torch.Tensor) -> torch.Tensor:
    if output_tensor.numel() == 0:
        return output_tensor
    # 如果是1维，就保持原来的逻辑
    if output_tensor.dim() == 1:
        last_token = output_tensor[-1]
        index = output_tensor.size(0) - 1
        while index >= 0 and output_tensor[index] == last_token:
            index -= 1
        return output_tensor[:index+1]
    # 如果是二维，则沿着最后一个维度去除尾部重复，同时保持二维形式
    elif output_tensor.dim() == 2:
        last_token = output_tensor[0, -1]
        index = output_tensor.size(1) - 1
        while index >= 0 and output_tensor[0, index] == last_token:
            index -= 1
        return output_tensor[:, :index+1]
    else:
        raise ValueError("仅支持1维和2维的tensor")

In [18]:
layer=17

results_all = []

from datasets import Dataset, load_dataset
dataset = load_dataset("json", data_files="./data/safety/gsm/test.jsonl", split="train")
all_ques = dataset[1:2]["question"]

for i, ques in enumerate(all_ques):
    results={}
    results["dataset"] = "gsm8k"
    results["question"] = ques
    results["id"] = i
    for multiplier in range(-3,4):
        caa_vector_path = os.path.join(
            "./data/r1/overthink_qwen/caa_vector/deepseek-qwen-7b_r1", f"{layer}.pt"
        )
        caa_vector = torch.load(caa_vector_path).to(device)
        # print("Caa vector path: ",caa_vector_path)
        # print("Caa vector: ",caa_vector)
        # print("caa vector: ",torch.norm(caa_vector, p=2))
        model.reset_all()
        model.set_add_activations(
            layer, multiplier * caa_vector
        )

        chat = [
            { "role": "user", "content": ques},
        ]
        prompt_tokens = tokenizer.apply_chat_template(chat, tokenize=True, temperature=0.1, add_generation_prompt=True, return_tensors="pt")
        # print(prompt_tokens)
        # print(f"{tokenizer.decode(prompt_tokens[0])}")

        prompt_tokens = prompt_tokens.to(device)  
        outputs={}
        token_lens={}

        for i in range(10):
            output = model.model.generate(prompt_tokens, max_new_tokens=1000)
            print(tokenizer.batch_decode(output))
            output = output[:,prompt_tokens.shape[-1]:]
            # print(output.shape)
            token_len_b = output.shape[-1]

            output = trim_trailing_duplicates(output)
            # print(output.shape)
            token_len_a = output.shape[-1]
            output = tokenizer.batch_decode(output)
            # print(output[0])
            outputs[i] = output[0]
            token_lens[i] = (token_len_b, token_len_a)

        results[multiplier] = {"outputs": outputs, "token_lens": token_lens}
    results_all.append(results)
import json
with open('./data/r1/overthink_qwen/results/results_qwen_ques_gsm_test.json', 'w') as json_file:
    json.dump(results_all, json_file, indent=4)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


["<｜begin▁of▁sentence｜><｜User｜>A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<｜Assistant｜><think>\nI need to figure out how much fiber the robe takes in total. It takes blue fiber and white fiber separately. The blue fiber is 2 bolts, and white fiber is half that much. So maybe I should start by thinking about how much white fiber is equivalent to the blue fiber. If blue is 2, then white is 2 bolts divided by 2, which is half. So maybe I can think in terms of comparing the two. Then I can add them up. If I have 2 bolts of blue and 2 of white, that's 2 more than the blue, which is half. So maybe the total is more than the blue, but how to compare? Maybe I need to make it in terms of something that's not a fraction. Maybe I can use the white fiber as a reference. The white is half the blue, but the blue is not the same as the white. Maybe I can make it like a ratio or something. I can think about the total in terms of how the whi

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


["<｜begin▁of▁sentence｜><｜User｜>A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<｜Assistant｜><think>\nI need to figure out how much fiber the robe takes in total. Maybe the problem is about combining two different types of fibers, blue and white. The blue fiber is 2 bolts, and white is half that much, which is less. Maybe I need to combine them into one total. If I take the blue and add the white, maybe it's like adding two different things that are not the same. Maybe they're not the same kind of fiber, but the problem is about the total in terms of the number of bolts. Maybe the total is more than just adding up two things. But maybe it's a trick question, and they're not really comparable. Maybe the two different fibers are different kinds of materials, so I can't just add them up. Maybe I have to think about it differently. I don't know, maybe the problem is a bit confusing. Maybe it's like the total fiber in the robe, but it'

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

['<｜begin▁of▁sentence｜><｜User｜>A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<｜Assistant｜><think>\nTo determine the total number of bolts in a comprehensive calculation, the following steps are as follows:\n\n1. **Identify the quantity of blue fiber:**\n   - 12 units.\n\n2. **Calculate the quantity of white fiber:**\n   - 12.<｜end▁of▁sentence｜>']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


['<｜begin▁of▁sentence｜><｜User｜>A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<｜Assistant｜><think>\nTo solve the problem, we start by identifying the quantities of blue and white fibers required.\n\n1. The problem states that 1000 units of blue fiber are required.\n\n2. The problem states that 1000 units of white fiber are required.\n\n3. To determine the decimal representation of 1000 is 1000.\n\n4. The numerical representation of 1000 is 1000.\n</think>\n\n**Solution:**\n\n1. **Identify the quantities:**\n\n   - **Blue Fiber:** 1000\n   - **White Fiber:** 1000<｜end▁of▁sentence｜>']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


['<｜begin▁of▁sentence｜><｜User｜>A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<｜Assistant｜><think>\nTo begin, establish the quantities of each color in the equation.<｜end▁of▁sentence｜>']
['<｜begin▁of▁sentence｜><｜User｜>A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<｜Assistant｜><think>\nTo determine the total number of bolts in a rope, follow these steps:\n\n1. **Identify the quantity of each color:**  \n   - **Blue:** 2  \n   - **White:** 2<｜end▁of▁sentence｜>']


In [17]:
for i in range(-3,4):
    print(i)

-3
-2
-1
0
1
2
3


In [4]:
results

{0: {'outputs': {0: "First, I need to determine the amount of blue fiber required for the robe, which is 2 bolts.\n\nNext, the amount of white fiber is half of the blue fiber, so I calculate half of 2 bolts, which equals 1 bolt.\n\nFinally, to find the total number of bolts, I add the blue and white fiber together: 2 bolts plus 1 bolt equals 3 bolts in total.\n</think>\n\nTo determine the total number of bolts needed for the robe, let's break down the requirements:\n\n1. **Blue Fiber:**\n   - The robe requires **2 bolts** of blue fiber.\n\n2. **White Fiber:**\n   - It requires **half** the amount of white fiber compared to blue fiber.\n   - Calculation: \\( \\frac{1}{2} \\times 2 \\text{ bolts} = 1 \\text{ bolt} \\)\n\n3. **Total Bolts:**\n   - Add the blue and white fibers together:\n   \\[\n   2 \\text{ bolts (blue)} + 1 \\text{ bolt (white)} = 3 \\text{ bolts in total}\n   \\]\n\n**Final Answer:** \\(\\boxed{3}\\)",
   1: "To determine the total number of bolts needed for the robe, 

In [8]:
results_all = []
results_all.append(results)
results_all

[{0: {'outputs': {0: "First, I need to determine the amount of blue fiber required for the robe, which is 2 bolts.\n\nNext, the amount of white fiber is half of the blue fiber, so I calculate half of 2 bolts, which equals 1 bolt.\n\nFinally, to find the total number of bolts, I add the blue and white fiber together: 2 bolts plus 1 bolt equals 3 bolts in total.\n</think>\n\nTo determine the total number of bolts needed for the robe, let's break down the requirements:\n\n1. **Blue Fiber:**\n   - The robe requires **2 bolts** of blue fiber.\n\n2. **White Fiber:**\n   - It requires **half** the amount of white fiber compared to blue fiber.\n   - Calculation: \\( \\frac{1}{2} \\times 2 \\text{ bolts} = 1 \\text{ bolt} \\)\n\n3. **Total Bolts:**\n   - Add the blue and white fibers together:\n   \\[\n   2 \\text{ bolts (blue)} + 1 \\text{ bolt (white)} = 3 \\text{ bolts in total}\n   \\]\n\n**Final Answer:** \\(\\boxed{3}\\)",
    1: "To determine the total number of bolts needed for the robe

In [6]:
import json
with open('./data/r1/overthink_qwen/results/results_qwen_ques_gsm1.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

In [3]:
a = []


for i in range(5):
    b = {}
    b["id"] = i
    a.append(b)
print(a)
b["id"] = 6
a

[{'id': 0}, {'id': 1}, {'id': 2}, {'id': 3}, {'id': 4}]


[{'id': 0}, {'id': 1}, {'id': 2}, {'id': 3}, {'id': 6}]